In [1]:
# Run once at top of notebook
!pip install -q -U transformers datasets accelerate peft bitsandbytes safetensors


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 99.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.4 MB/s eta 0:

In [2]:
import json
from datasets import Dataset
from pathlib import Path

TRAIN_JSON_PATH = "/kaggle/input/allcombined/Untitled-1.json"   # <-- change to your file path if different
MAX_LENGTH = 512   # tokens (adjust if needed)

# Load raw JSON list
with open(TRAIN_JSON_PATH, "r", encoding="utf-8") as f:
    raw = json.load(f)

# Normalize / build prompt/response strings
examples = []
for item in raw:
    # safeguard field names: uses question & answer fields as user showed
    q = item.get("question", "").strip()
    a = item.get("answer", "").strip()
    # you can include metadata if you want (source_pdf, page)
    meta = f" [source: {item.get('source_pdf','')}, page: {item.get('page','')}]"
    # Compose training text — you can change format to your instruction style
    text = f"### Question:\n{q}\n\n### Answer:\n{a}{meta}"
    examples.append({"text": text})

ds = Dataset.from_list(examples)
print("Built dataset with", len(ds), "examples")


Built dataset with 1504 examples


In [3]:
from transformers import AutoTokenizer
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# If you saved locally earlier, point to local folder. Otherwise use HF ID.

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
# ensure pad token exists for Trainer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, max_length=MAX_LENGTH, padding="max_length")

tokenized = ds.map(tokenize_fn, batched=True, remove_columns=["text"])
tokenized.set_format(type="torch")
print("Tokenized dataset example:", tokenized[0]["input_ids"].shape)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/1504 [00:00<?, ? examples/s]

Tokenized dataset example: torch.Size([512])


In [4]:
import torch
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if DEVICE == "cuda" else torch.float32

print("Device:", DEVICE, "dtype:", dtype)

# Load base model (no quantization) — set device_map="auto" for multi-GPU or bigger
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=dtype,
    device_map="auto"
)

# Prepare for k-bit training if you use 8/4-bit; if not using bitsandbytes it's safe too.
# prepare_model_for_kbit_training(model)  # uncomment only if you're using k-bit quantization

# LoRA config
lora_config = LoraConfig(
    r=8,                       # rank
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # cover common proj names
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # verify only LoRA params are trainable


2025-10-30 07:59:21.627114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761811161.857545      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761811161.917325      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Device: cuda dtype: torch.float16


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

trainable params: 2,179,072 || all params: 1,779,267,072 || trainable%: 0.1225


In [5]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

OUTPUT_DIR = "/kaggle/working/deepseek-1.5b-physics-lora"
per_device_batch = 1
grad_accum = 8   # effective batch = 8

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=per_device_batch,
    gradient_accumulation_steps=grad_accum,
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=(DEVICE=="cuda"),
    logging_steps=50,
    save_strategy="epoch",
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


/tmp/ipykernel_37/2494954169.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [6]:
# Train! Might take minutes → hours depending on dataset size and epochs
trainer.train()
# Save final
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ Fine-tuned LoRA adapters saved to:", OUTPUT_DIR)


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 151646, 'pad_token_id': 151643}.


Step,Training Loss
50,2.918800
100,1.929700
150,1.659200
200,1.559000
250,1.519100
300,1.478300
350,1.470500


✅ Fine-tuned LoRA adapters saved to: /kaggle/working/deepseek-1.5b-physics-lora


In [11]:
!tar -czf /kaggle/working/deepseek-1.5b-physics-lora.tar.gz -C /kaggle/working deepseek-1.5b-physics-lora


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
!pip install -q "transformers>=4.43,<4.46" "sentence-transformers>=3.0,<3.2" "huggingface_hub>=0.26,<0.28"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 73.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 62.6 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.38.1 requires huggingface-hub>=0.28.1, but you have huggingface-hub 0.27.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.


In [3]:
!pip install -q faiss-cpu pymupdf pillow pytesseract nltk

# download punkt tokenizer for sentence splitting

import nltk
nltk.download('punkt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 56.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 71.7 MB/s eta 0:00:00:00:0100:01


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import os
import io
import time
import math
import json
import faiss
import fitz            # PyMuPDF
import torch
import pytesseract
from PIL import Image
from pathlib import Path
from typing import List, Tuple
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import transformers.utils.hub as hf_hub_utils

/usr/local/lib/python3.11/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-10-30 17:26:04.533327: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761845164.912372      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761845165.019298      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
hf_hub_utils.list_repo_templates = lambda *args, **kwargs: []

# Device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

Device: cuda


In [18]:
from peft import PeftModel
BASE_MODEL = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
LORA_ADAPTER_PATH = "/kaggle/working/deepseek-1.5b-physics-lora"  # <-- your fine-tuned adapter path
MERGED_MODEL_PATH = "/kaggle/working/deepseek-physics-finetuned"  # <-- where to save merged model

# 1. Load base model in fp16 if GPU is available
dtype = torch.float16 if torch.cuda.is_available() else torch.float32
device = "cuda" if torch.cuda.is_available() else "cpu"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=dtype,
    device_map="auto"
)

# 2. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)

# 3. Load LoRA adapter weights
print("🔧 Loading LoRA adapter...")
lora_model = PeftModel.from_pretrained(base_model, LORA_ADAPTER_PATH)

# 4. Merge LoRA into base weights
print("🧠 Merging LoRA adapter into base model weights...")
merged_model = lora_model.merge_and_unload()   # <-- this creates a standalone model

# 5. Save merged model and tokenizer
print(f"💾 Saving merged model to {MERGED_MODEL_PATH}")
merged_model.save_pretrained(MERGED_MODEL_PATH)
tokenizer.save_pretrained(MERGED_MODEL_PATH)

print("✅ Merged model saved successfully!")

🔧 Loading LoRA adapter...
🧠 Merging LoRA adapter into base model weights...
💾 Saving merged model to /kaggle/working/deepseek-physics-finetuned
✅ Merged model saved successfully!


In [16]:
PDF_INPUT_PATH = "/kaggle/input/pdf-file/Angew Chem Int Ed - 2023 - Glico - Toward MagnetoOptical Cryogenic Thermometers with High Sensitivity  A Magnetic.pdf"

# Model choices
EMB_MODEL_NAME = "all-mpnet-base-v2"  # stronger semantic embeddings
HF_GEN_MODEL = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"  # generator

# Chunking / retrieval params
CHUNK_SIZE_WORDS = 400
CHUNK_OVERLAP_WORDS = 80
TOP_K = 5

# Caching (optional)
CACHE_DIR = "/kaggle/working/rag_cache"
os.makedirs(CACHE_DIR, exist_ok=True)
EMB_CACHE_FNAME = os.path.join(CACHE_DIR, "embeddings.npy")
CHUNKS_CACHE_FNAME = os.path.join(CACHE_DIR, "chunks.json")
FAISS_INDEX_FNAME = os.path.join(CACHE_DIR, "faiss.index")
EMB_MODEL_CACHE = os.path.join(CACHE_DIR, "sbert")  # optional SBERT cache path

# 3) PDF extraction (text + images OCR + simple "table-like" extraction)
# ============================
import re
from nltk.tokenize import sent_tokenize

def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extracts text from a PDF including:
      - textual content via PyMuPDF
      - "table-like" blocks detected by tabs or multiple consecutive spaces
      - OCR from embedded images via pytesseract
    Returns the combined text.
    """
    doc = fitz.open(pdf_path)
    page_texts = []
    for pno, page in enumerate(doc, start=1):
        # 1) regular text (flow text)
        text = page.get_text("text") or ""
        
        # 2) table-like heuristics: detect blocks with multiple tabs / consecutive multi-space
        blocks = page.get_text("blocks")
        table_texts = []
        for b in blocks:
            block_text = b[4].strip()
            # heuristics: tabs or many consecutive spaces (possible table or fixed-width)
            if "\t" in block_text or re.search(r" {2,}", block_text):
                table_texts.append(block_text)
        if table_texts:
            text += "\n\n" + "\n".join(table_texts)
        
        # 3) images -> OCR
        images = page.get_images(full=True)
        for img_index, img in enumerate(images):
            xref = img[0]
            try:
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                pil = Image.open(io.BytesIO(image_bytes)).convert("RGB")
                ocr = pytesseract.image_to_string(pil)
                if ocr and ocr.strip():
                    text += "\n\n[OCR PAGE %d IMAGE %d]\n%s" % (pno, img_index, ocr)
            except Exception:
                # continue without crashing if an image fails
                continue
        if text.strip():
            page_texts.append(f"[PAGE {pno}]\n" + text.strip())
    doc.close()
    return "\n\n".join(page_texts)

# Quick check: ensure file exists
if not os.path.isfile(PDF_INPUT_PATH):
    raise FileNotFoundError(f"PDF not found at {PDF_INPUT_PATH}. Run !ls /kaggle/input to check path.")
print("PDF found — ready to extract.")

PDF found — ready to extract.


In [15]:
def chunk_text(text: str, chunk_size_words: int = CHUNK_SIZE_WORDS, overlap_words: int = CHUNK_OVERLAP_WORDS) -> List[str]:
    """
    Splits text into overlapping chunks based on sentences.
    - chunk_size_words: approx target words per chunk
    - overlap_words: words to keep overlapping between consecutive chunks
    """
    # split into sentences (better than naive ". ")
    sentences = sent_tokenize(text)
    chunks = []
    cur_chunk = []
    cur_words = 0
    for sent in sentences:
        sent_words = len(sent.split())
        if cur_words + sent_words > chunk_size_words and cur_chunk:
            # flush
            chunk_text = " ".join(cur_chunk).strip()
            if len(chunk_text.split()) >= 20:
                chunks.append(chunk_text)
            # start new chunk with overlap
            # keep last overlap_words of current chunk as new start
            words_keep = []
            accum = 0
            # iterate sentences in cur_chunk reversed to keep overlap
            for s in reversed(cur_chunk):
                w = s.split()
                if accum + len(w) > overlap_words:
                    break
                words_keep.insert(0, s)
                accum += len(w)
            cur_chunk = words_keep.copy()
            cur_words = sum(len(s.split()) for s in cur_chunk)
        cur_chunk.append(sent)
        cur_words += sent_words
    # final flush
    if cur_chunk:
        chunks.append(" ".join(cur_chunk).strip())
    return chunks

In [7]:
# ============================
# 5) Build embeddings & FAISS (cosine / inner-product) with caching
# ============================
import numpy as np

def build_or_load_index(chunks: List[str], emb_model_name: str = EMB_MODEL_NAME):
    # If cached files exist, load them
    if os.path.exists(CHUNKS_CACHE_FNAME) and os.path.exists(EMB_CACHE_FNAME) and os.path.exists(FAISS_INDEX_FNAME):
        print("Loading cached chunks, embeddings and FAISS index from disk...")
        with open(CHUNKS_CACHE_FNAME, "r", encoding="utf8") as f:
            cached_chunks = json.load(f)
        embeddings = np.load(EMB_CACHE_FNAME)
        index = faiss.read_index(FAISS_INDEX_FNAME)
        # load SBERT model (for query embedding)
        sbert = SentenceTransformer(emb_model_name)
        return cached_chunks, index, embeddings, sbert

    # Build afresh
    print("Loading SBERT model for embeddings:", emb_model_name)
    sbert = SentenceTransformer(emb_model_name)
    print("Encoding chunks (this can take a while)...")
    embeddings = sbert.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
    # normalize embeddings for cosine similarity (IndexFlatIP)
    faiss.normalize_L2(embeddings)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    # persist
    print("Caching chunks, embeddings and FAISS index to disk...")
    with open(CHUNKS_CACHE_FNAME, "w", encoding="utf8") as f:
        json.dump(chunks, f)
    np.save(EMB_CACHE_FNAME, embeddings)
    faiss.write_index(index, FAISS_INDEX_FNAME)
    return chunks, index, embeddings, sbert


In [14]:
def load_generator_model(model_name: str = HF_GEN_MODEL, lora_path: str = None):
    print(f"🔍 Loading base model: {model_name}")
    dtype = torch.float16 if DEVICE == "cuda" else torch.float32

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, trust_remote_code=True)

    # Load base model on a single GPU (force cuda:0)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=dtype,
        low_cpu_mem_usage=True,
                # ⚠️ turn off auto device mapping
        trust_remote_code=True
    ).to(DEVICE)                # ✅ explicitly move to one device

    # If LoRA fine-tuned weights are provided
    if lora_path:
        
        model = PeftModel.from_pretrained(model, lora_path)
        model.to(DEVICE)        # ✅ ensure LoRA adapter is also on same device
    
        

    model.eval()
    
    return tokenizer, model

 
def generate_with_context(question: str, retrieved_chunks: List[str], tokenizer, model,
                          max_new_tokens: int = 512, temperature: float = 0.0, top_p: float = 0.95):
    """
    Assembles the prompt and generates an answer using the LLM.
    The prompt instructs the model to answer only from provided context and to say 'Not found.' if not present.
    """
    # Add short source labels to each chunk for traceability
    labeled = []
    for i, c in enumerate(retrieved_chunks, start=1):
        snippet = c if len(c) < 1000 else c[:1000] + " ...[truncated]"
        labeled.append(f"[DOC {i}]\n{snippet}")
    context = "\n\n".join(labeled)

    prompt = (
        "You are an assistant that answers questions using ONLY the provided document excerpts.\n"
        "If the answer is not present in the excerpts, reply exactly: Not found.\n"
        "Cite the source by DOC number if providing facts.\n\n"
        f"CONTEXT:\n{context}\n\nQUESTION:\n{question}\n\nANSWER:"
    )

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to(DEVICE)
    gen_cfg = GenerationConfig(max_new_tokens=max_new_tokens, temperature=temperature, top_p=top_p)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=gen_cfg.max_new_tokens,
                                 do_sample=(temperature > 0.0), temperature=gen_cfg.temperature,
                                 top_p=gen_cfg.top_p)
    generated = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
    return generated

In [17]:
# ============================
# 7) Full pipeline run: extract -> chunk -> index -> interactive Q&A
# ============================
# 7.1 Extract text
print("Extracting text from PDF (this may take some time for large PDFs)...")
full_text = extract_text_from_pdf(PDF_INPUT_PATH)
print("Extracted text length (chars):", len(full_text))

# 7.2 Chunk text
print("Chunking text into sentence-based chunks...")
chunks = chunk_text(full_text, chunk_size_words=CHUNK_SIZE_WORDS, overlap_words=CHUNK_OVERLAP_WORDS)
print("Number of chunks:", len(chunks))

# 7.3 Build or load FAISS
chunks, faiss_index, embeddings, emb_model = build_or_load_index(chunks, EMB_MODEL_NAME)
print("FAISS index has", faiss_index.ntotal, "vectors.")

# 7.4 Load generator
tokenizer, generator_model = load_generator_model(HF_GEN_MODEL,"/kaggle/working/deepseek-1.5b-physics-lora")

# 7.5 Interactive loop (you can also script queries)
def answer_query(question: str, top_k: int = TOP_K):
    # embed question
    q_emb = emb_model.encode([question], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    D, I = faiss_index.search(q_emb, top_k)
    retrieved = [chunks[i] for i in I[0] if i != -1]
    # show retrieved chunk indices & scores (D are similarity scores)
    print("\n--- Retrieved (top_k={}): ---".format(top_k))
    for rank, (idx, score) in enumerate(zip(I[0], D[0]), start=1):
        if idx == -1: continue
        print(f"Rank {rank} — chunk_id={idx} — score={score:.4f} — preview: {repr(chunks[idx][:200])}")

    # generate
    ans = generate_with_context(question, retrieved, tokenizer, generator_model)
    return ans

# Example query (change as needed)
example_q = "What happens on increasing magnetic fields?"
print("\nExample query:", example_q)
print("Answer:\n", answer_query(example_q, top_k=TOP_K))


Extracting text from PDF (this may take some time for large PDFs)...
Extracted text length (chars): 34903
Chunking text into sentence-based chunks...
Number of chunks: 18
Loading cached chunks, embeddings and FAISS index from disk...
FAISS index has 18 vectors.
🔍 Loading base model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B

Example query: What happens on increasing magnetic fields?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configurati


--- Retrieved (top_k=5): ---
Rank 1 — chunk_id=9 — score=0.2473 — preview: 'In this scheme the enhancement of the rcp component and consequently, the band asymmetry is exaggerated in order to\nfacilitate the understanding of the effect. Angewandte\nChemie\nCommunications\nAngew. '
Rank 2 — chunk_id=7 — score=0.2247 — preview: '—_—_ °G,\n205 — tas *Ke\n—<—<—<— °G,\n——_ 3H,\n254\n— 5G,\n— °F,\n5\n—_— °F,\n20, °F\nil °F, 5S,\n154 —— FFs\n14\no4 — 5\n\n700\n\n[PAGE 4]\nconsiderable intensities (three positive and three negative)\npossibilities th'
Rank 3 — chunk_id=11 — score=0.2240 — preview: 'Variable-temperature MCD spectra for HoIII complex under a 0.25 T applied magnetic field. (a) VT MCD spectra for HoIII complex. (b)\nTemperature dependence of the different Δ parameters. (c) Relative s'
Rank 4 — chunk_id=6 — score=0.1934 — preview: 'Besides the outstanding results obtained with the NdIII\nand DyIII complexes, we also studied the magneto-optical\nthermometric performance of the HoII

In [21]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()